Duvall Pinkney New York State labor market

# Homework 20

We will use the New York State labor market data set from Homework 19.  

Data URL:  [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/labor-market-Feb2021.csv](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/labor-market-Feb2021.csv)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

from sklearn.cluster import KMeans

from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_score

from sklearn.datasets import make_blobs

%matplotlib inline
pd.set_option("display.max_columns", None)

In [3]:
url = "https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/labor-market-Feb2021.csv"
df = pd.read_csv(url)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,FEDERAL RESERVE BANK of NEW YORK,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
84,Engineering Technologies,3.1,45.7,"50,000","85,000",24.9
85,Miscellaneous Technologies,3.2,52.2,"45,000","80,000",19.0
86,Overall,3.6,42.5,"45,000","70,000",37.9
87,NaN,NaN,NaN,NaN,NaN,NaN


### Question 1: Load, clean and scale the data 

This question combines Question 1 of Homework 19 with scaling the data (Questions 2a and 3a of Homework 19).

a) This data file requires some cleaning. Open the CSV file in Jupyter or another text editor to see what the possible problems are, and refer to Lab 4 for how to fix them.

In [5]:
df.dropna()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
12,Major,Unemployment Rate,Underemployment Rate,Median Wage Early Career,Median Wage Mid-Career,Share with Graduate Degree
13,Agriculture,3.0,60.3,"40,000","65,000",20.8
14,Animal and Plant Sciences,2.6,58.6,"37,000","63,000",35.1
15,Environmental Studies,3.9,53.7,"39,000","65,000",30.5
16,Architecture,2.2,27.9,"50,000","76,000",41.0
...,...,...,...,...,...,...
82,International Affairs,4.1,53.1,"46,150","79,000",43.0
83,History,5.5,53.6,"40,000","64,000",49.5
84,Engineering Technologies,3.1,45.7,"50,000","85,000",24.9
85,Miscellaneous Technologies,3.2,52.2,"45,000","80,000",19.0


1b) Create a new DataFrame with the 5 quantitative columns (Unemployment Rate,Underemployment Rate,Median Wage Early Career,Median Wage Mid-Career,Share with Graduate Degree) scaled to be between 0 and 1.

In [8]:
df2 = pd.get_dummies(df)
df2

,Unnamed: 0_Accounting,Unnamed: 0_Advertising and Public Relations,Unnamed: 0_Aerospace Engineering,Unnamed: 0_Agriculture,Unnamed: 0_Animal and Plant Sciences,Unnamed: 0_Anthropology,Unnamed: 0_Architecture,Unnamed: 0_Art History,Unnamed: 0_Biochemistry,Unnamed: 0_Biology,Unnamed: 0_Business Analytics,Unnamed: 0_Business Management,"Unnamed: 0_Chart and Table Data for Federal Reserve Bank of New York, The Labor Market for Recent College Graduates interactive web feature",Unnamed: 0_Chemical Engineering,Unnamed: 0_Chemistry,Unnamed: 0_Civil Engineering,Unnamed: 0_Commercial Art & Graphic Design,Unnamed: 0_Communications,Unnamed: 0_Computer Engineering,Unnamed: 0_Computer Science,Unnamed: 0_Construction Services,Unnamed: 0_Criminal Justice,Unnamed: 0_Early Childhood Education,Unnamed: 0_Earth Sciences,Unnamed: 0_Economics,Unnamed: 0_Electrical Engineering,Unnamed: 0_Elementary Education,Unnamed: 0_Engineering Technologies,Unnamed: 0_English Language,Unnamed: 0_Environmental Studies,Unnamed: 0_Ethnic Studies,Unnamed: 0_FEDERAL RESERVE BANK of NEW YORK,Unnamed: 0_Family and Consumer Sciences,Unnamed: 0_Finance,Unnamed: 0_Fine Arts,Unnamed: 0_Foreign Language,Unnamed: 0_General Business,Unnamed: 0_General Education,Unnamed: 0_General Engineering,Unnamed: 0_General Social Sciences,Unnamed: 0_Geography,Unnamed: 0_Health Services,Unnamed: 0_History,Unnamed: 0_Industrial Engineering,Unnamed: 0_Information Systems & Management,Unnamed: 0_Interdisciplinary Studies,Unnamed: 0_International Affairs,Unnamed: 0_Journalism,Unnamed: 0_Leisure and Hospitality,Unnamed: 0_Liberal Arts,Unnamed: 0_Major,Unnamed: 0_Marketing,Unnamed: 0_Mass Media,Unnamed: 0_Mathematics,Unnamed: 0_Mechanical Engineering,Unnamed: 0_Medical Technicians,Unnamed: 0_Miscellaneous Biological Science,Unnamed: 0_Miscellaneous Education,Unnamed: 0_Miscellaneous Engineering,Unnamed: 0_Miscellaneous Physical Sciences,Unnamed: 0_Miscellaneous Technologies,Unnamed: 0_Nursing,Unnamed: 0_Nutrition Sciences,Unnamed: 0_Overall,Unnamed: 0_Performing Arts,Unnamed: 0_Pharmacy,Unnamed: 0_Philosophy,Unnamed: 0_Physics,Unnamed: 0_Political Science,Unnamed: 0_Psychology,Unnamed: 0_Public Policy and Law,Unnamed: 0_Secondary Education,Unnamed: 0_Social Services,Unnamed: 0_Sociology,"Unnamed: 0_Sources: U.S. Census Bureau, American Community Survey (IPUMS); U.S. Department of Labor, O*NET.",Unnamed: 0_Special Education,Unnamed: 0_Theology and Religion,Unnamed: 0_Treatment Theraphy,"Unnamed: 0_Updated: February 12, 2021",Unnamed: 1_1.2,Unnamed: 1_1.3,Unnamed: 1_1.5,Unnamed: 1_1.7,Unnamed: 1_1.9,Unnamed: 1_2.1,Unnamed: 1_2.2,Unnamed: 1_2.3,Unnamed: 1_2.5,Unnamed: 1_2.6,Unnamed: 1_2.9,Unnamed: 1_3.0,Unnamed: 1_3.1,Unnamed: 1_3.2,Unnamed: 1_3.3,Unnamed: 1_3.4,Unnamed: 1_3.5,Unnamed: 1_3.6,Unnamed: 1_3.7,Unnamed: 1_3.8,Unnamed: 1_3.9,Unnamed: 1_4.0,Unnamed: 1_4.1,Unnamed: 1_4.2,Unnamed: 1_4.4,Unnamed: 1_4.6,Unnamed: 1_4.7,Unnamed: 1_4.8,Unnamed: 1_4.9,Unnamed: 1_5.1,Unnamed: 1_5.4,Unnamed: 1_5.5,Unnamed: 1_5.8,Unnamed: 1_6.1,Unnamed: 1_6.2,Unnamed: 1_6.3,Unnamed: 1_8.0,Unnamed: 1_8.5,Unnamed: 1_Unemployment Rate,Unnamed: 2_12.1,Unnamed: 2_13.6,Unnamed: 2_15.5,Unnamed: 2_16.4,Unnamed: 2_17.8,Unnamed: 2_18.0,Unnamed: 2_18.4,Unnamed: 2_19.8,Unnamed: 2_21.4,Unnamed: 2_21.7,Unnamed: 2_21.8,Unnamed: 2_21.9,Unnamed: 2_22.9,Unnamed: 2_23.1,Unnamed: 2_23.7,Unnamed: 2_25.0,Unnamed: 2_25.8,Unnamed: 2_27.1,Unnamed: 2_27.9,Unnamed: 2_28.3,Unnamed: 2_28.4,Unnamed: 2_31.0,Unnamed: 2_31.5,Unnamed: 2_31.7,Unnamed: 2_32.4,Unnamed: 2_34.3,Unnamed: 2_35.2,Unnamed: 2_36.4,Unnamed: 2_37.6,Unnamed: 2_40.6,Unnamed: 2_40.8,Unnamed: 2_41.6,Unnamed: 2_41.7,Unnamed: 2_42.5,Unnamed: 2_42.8,Unnamed: 2_44.7,Unnamed: 2_45.3,Unnamed: 2_45.7,Unnamed: 2_45.9,Unnamed: 2_47.6,Unnamed: 2_47.9,Unnamed: 2_48.4,Unnamed: 2_49.5,Unnamed: 2_50.1,Unnamed: 2_50.5,Unnamed: 2_50.8,Unnamed: 2_50.9,Unnamed: 2_51.2,Unnamed: 2_51.5,Unnamed: 2_51.6,Unnamed: 2_51.9,Unnamed: 2_52.1,Unnamed: 2_52.2,Unnamed: 2_52.3,Unnamed: 2_53.1,Unnamed: 2_53.5,Unname

### Question 2:  Elbow method

2a) Write a loop that uses the k-means algorithm to cluster the scaled labor data for each k from 1 to 10, and store the inertia for each cluster in a list.

2b) Plot a line plot of the inertias for each k.  Based on this plot, what how many clusters do you think this data contains?

2c) Re-run k-means with the k you chose in Question 2b, and plot the scatterplots of all pairs of quantitative variables in the original labor DataFrame colored by the cluster.  Based on this visualization, how good do you think this clustering is?  Are there any problems?    

### Question 3: Silhouette Method

a) Write a loop that uses the k-means algorithm to cluster the scaled labor data for each k from 2 to 10, and store the silhouette score for each cluster in a list.

3b) Plot a line plot of the silhouette scores for each k.  Based on this plot, what how many clusters do you think this data contains?

3c) Re-run k-means with the k you chose in Question 3b, and plot the scatterplots of all pairs of quantitative variables in the original labor DataFrame colored by the cluster. Based on this visualization, how good do you think this clustering is? Are there any problems?